In [1]:
import matplotlib.pyplot as plt
import transformers
import pandas as pd
import numpy as np
import torch
import json
import sys
import os

from dotenv import load_dotenv
from huggingface_hub import login
from transformers import pipeline

In [2]:
load_dotenv()
path = os.environ['DATA_PATH']
os.environ["ACCELERATE_USE_TORCH_DEVICE"] = "true"
login(token=os.environ['HF_TOKEN'])
cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\Desktop\Università\MNLP\Homework2\cache\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


<|system|>
You are a chat bot that translates the message received from italian to english</s>
<|user|>
Roma è in centro italia</s>
<|assistant|>
Roma è in centro italiano.


We use the tokenizer's chat template to format each message
 
Refer to https://huggingface.co/docs/transformers/main/en/chat_templating

In [19]:
# prompt = [
#     {
#         "role": "system",
#         "content": "You are a chat bot that translates the message received from italian to english",
#     },
#     {"role": "user", "content": "Roma è in centro italia"},
# ]

prompt = """
### Instruction: Translate the following italian sentence into english: roma è al centro dell' Italia
### Response: 
"""

#prompt = pipe.tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


### Instruction: Translate the following italian sentence into english: roma è al centro dell' Italia
### Response: 
Roma, the capital of Italy, is located in the center of Italy.


In [20]:
datasetRaw = {}

for i in [ocred_fn, cleaned_fn]:
    if i not in os.listdir(path):
        print(f"ERROR 404 ! File {i} not Found...")

    file_path = os.path.join(path, i)
    with open(file_path, 'r') as f:
        datasetRaw[i.split('.')[0]] = json.load(f)
        f.close()

In [21]:
sample = datasetRaw['original_ocr']['1']
question = f"""
### Instruction: This paragraph in italian contains noise, correct in: {sample}
### Response: 
"""

In [23]:
outputs = pipe(question, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (2170 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



### Instruction: This paragraph in italian contains noise, correct in: I. 
Como  andò  che  Maestro  Ciliegia,  Megnamc 
trovò  un  pezzo  di  legno  che  piangeva  e  rideva  come  un  bambino. 
—  C'era  una  volta.... 
—  Un  re!  -diranno  subito  i  miei  piccoli  lettori. 
—  Ko,  ragazzi,  avete  sbagliato.  C'era  una  volta 
un  pezzo  di  legno. 
Kon  era  un  legno  di  lusso,  ma  un  semplice 
pezzo  da  catasta,  di  quelli  che  d' inverno  si  met- 
tono nelle  stufe  e  nei  caminetti  per  accendere 
il  fuoco  e  per  riscaldare  le  stanze. 

Non  so  come  andasse,  ma  il  fatto  gli  è  che  un 
bel  giorno' questo  pezzo  di  legno  capitò  nella  bot- 
tega di  un  vecchio  falegname,  il  quale  aveva 
nome  mastr' Antonio,  se  non  che  tutti  lo  chia- 
mavano maestro  Ciliegia,  per  via  della  punta  del 

sentì  nna  vocina  sottile  sottile. 

SUO  naso,  che  era  sempre  lustra  e  paonazza,  come 
una  ciliegia  matura. 
Appena  maestro  Ciliegia  